In [1]:
# Install library for reading JPK File
%pip uninstall jpkreader --yes
%pip install git+https://github.com/jlopezalo/pyjpkreader

Found existing installation: jpkreader 0.0.1
Uninstalling jpkreader-0.0.1:
  Successfully uninstalled jpkreader-0.0.1
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/jlopezalo/pyjpkreader to /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-ezuy7sfb
  Running command git clone -q https://github.com/jlopezalo/pyjpkreader /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-ezuy7sfb
  Created wheel for jpkreader: filename=jpkreader-0.0.1-py3-none-any.whl size=10194 sha256=471dc74ac3450928583d82bfeeb9ea669893341286a93c200b4278e0a61c5e2d
  Stored in directory: /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-ephem-wheel-cache-g905so55/wheels/23/c0/8e/c378f28c65b0199f6092b6aa45e090c1ed6b4093f3adb03a15
Successfully built jpkreader
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Install library for AFM data Rheology Analysis
%pip uninstall pyafmrheo --yes
%pip install git+https://github.com/jlopezalo/pyafmrheo.git

Found existing installation: pyafmrheo 0.0.1
Uninstalling pyafmrheo-0.0.1:
  Successfully uninstalled pyafmrheo-0.0.1
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/jlopezalo/pyafmrheo.git to /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-ejnel06s
  Running command git clone -q https://github.com/jlopezalo/pyafmrheo.git /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-ejnel06s
  Created wheel for pyafmrheo: filename=pyafmrheo-0.0.1-py3-none-any.whl size=23325 sha256=de8cbba3d76447c24b198fd427848f92296e75b8086ee115da35ac3fec3cedf8
  Stored in directory: /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-ephem-wheel-cache-ff4684dn/wheels/fb/66/18/3e93b45c845b0d7b83ded1e127a1c127820d070ce5b71dece4
Successfully built pyafmrheo
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import libraries we will need
import numpy as np

# Get file reader from library
from jpkreader import load_jpk_file
import matplotlib.pyplot as plt

# Get data analysis tools
from pyafmrheo.utils.force_curves import *
from pyafmrheo.hertz_fit import HertzFit
from pyafmrheo.ting_numerical_fit import TingNumericalFit
from pyafmrheo.ting_analytical_fit import TingAnaliticalFit
from pyafmrheo.models.ting_numerical import simple_power_law

ModuleNotFoundError: No module named 'hertz'

In [ ]:
# Define global variables
file_path = '/Users/javierlopez/Documents/Marseille/05-11-2021/E/map-data-2021.11.05-15.41.45.699.jpk-force-map'
# Shapes available: Paraboloid, Pyramidal
indenter_shape = "pyramid"
# tip_parameter = 5000 * 1e-9 # meters
tip_parameter = 35 # degrees
poisson_ratio = 0.5
# If None it will use the deflection sensitivity from the file
deflection_sensitivity = None # m/V
# If None it will use the spring constant from the file
spring_constant = None # N/m
contact_offset= 0

In [ ]:
file = load_jpk_file(file_path)

In [ ]:
file_id = file.file_id
file_type = file.file_type
file_metadata = file.file_metadata
file_data = file.data
piezo_image = file.piezo_image
quality_map = file.quality_map

In [ ]:
closed_loop = file_metadata['z_closed_loop_status']
file_deflection_sensitivity = file_metadata['original_deflection_sensitivity'] #nm/V
file_spring_constant = file_metadata['original_spring_constant'] #N/m
height_channel = file_metadata['height_channel_key']

if not deflection_sensitivity: deflection_sensitivity = file_deflection_sensitivity / 1e9 #m/V
if not spring_constant: spring_constant = file_spring_constant

In [ ]:
curve_idx = 0
force_curve = file_data[curve_idx]
extend_segments = force_curve.extend_segments
pause_segments = force_curve.pause_segments
modulation_segments = force_curve.modulation_segments
retract_segments = force_curve.retract_segments
force_curve_segments = [*extend_segments, *pause_segments, *modulation_segments, *retract_segments]
force_curve_segments = sorted(force_curve_segments, key=lambda x: int(x[0]))

In [ ]:
t0 = 0
for seg_id, segment in force_curve_segments:
    height = segment.segment_formated_data[height_channel]
    time = segment.segment_formated_data["time"] + t0
    plt.plot(time, height)
    t0 = time[-1]

plt.xlabel("Time [s]")
plt.ylabel("Height [Meters]")
plt.grid()
plt.show()

In [ ]:
t0 = 0
for seg_id, segment in force_curve_segments:
    deflection = segment.segment_formated_data["vDeflection"]
    time = segment.segment_formated_data["time"] + t0
    plt.plot(time, deflection)
    t0 = time[-1]

plt.xlabel("Time [s]")
plt.ylabel("Deflection [Volts]")
plt.grid()
plt.show()

In [ ]:
for seg_id, segment in force_curve_segments:
    height = segment.segment_formated_data[height_channel]
    deflection = segment.segment_formated_data["vDeflection"]
    plt.plot(height, deflection)

plt.xlabel("Piezo Height [Meters]")
plt.ylabel("vDeflection [Volts]")
plt.grid()
plt.show()

In [ ]:
_, approach_segment = force_curve.extend_segments[0]
approach_segment_metadata = approach_segment.segment_metadata

app_deflection, app_height, app_time =\
    preprocess_segment(approach_segment, height_channel, deflection_sensitivity)

_, retract_segment = force_curve.retract_segments[-1]
retract_segment_metadata = retract_segment.segment_metadata
ret_deflection, ret_height, ret_time =\
    preprocess_segment(retract_segment, height_channel, deflection_sensitivity)

In [ ]:
# Shift height
xzero = ret_height[-1] # Maximum height
app_height = xzero - app_height
ret_height = xzero - ret_height

In [ ]:
# Find initial PoC, you may get some warnings.
poc = get_poc_RoV_method(app_height, app_deflection, win_size=100)
print(*poc)

In [ ]:
plt.plot(app_height, app_deflection)
plt.axvline(x=poc[0], color='r', linestyle='--')
plt.axhline(y=poc[1], color='r', linestyle='--')
plt.grid()
plt.show()

In [ ]:
# Hertz fit
poc[1] = 0

app_indentation, app_force =\
    get_force_vs_indentation_curve(app_height, app_deflection, poc, spring_constant)
ret_indentation, ret_force =\
    get_force_vs_indentation_curve(ret_height, ret_deflection, poc, spring_constant)

# Initial parameters
# delta0, f0, slope, E0
p0 = [0, 0, 1, 1000]

hertz_result = HertzFit(app_indentation, app_force, indenter_shape, tip_parameter, p0, poisson_ratio)

hertz_E = hertz_result.best_values['E0']
hertz_d0 = hertz_result.best_values['delta0']

print(hertz_result.fit_report())

# Plot Hertz Fit results
plt.plot(app_indentation, app_force)
plt.plot(app_indentation, hertz_result.best_fit)
plt.xlabel('Indentation [m]')
plt.ylabel('Force [N]')
plt.grid()
plt.show()

In [ ]:
# Get force vs indentation for all segments
poc[0] += hertz_d0
poc[1] = 0

# Get indentation and force
app_indentation, app_force = get_force_vs_indentation_curve(app_height, app_deflection, poc, spring_constant)
ret_indentation, ret_force = get_force_vs_indentation_curve(ret_height, ret_deflection, poc, spring_constant)

In [ ]:
indentation = np.r_[app_indentation, ret_indentation]
force = np.r_[app_force, ret_force]
t0 = app_time[-1]
time = np.r_[app_time, ret_time + t0]

In [ ]:
fit_mask = indentation > (-1 * contact_offset)
    
ind_fit = indentation[fit_mask] 
force_fit = force[fit_mask]
time_fit = time[fit_mask]
time_fit = time_fit - time_fit[0]

# Parameters for fit
dT = time_fit[1] - time_fit[0]
dTp = 1
E0 = hertz_E
fluidity_exponent = 0.15
smoothing_window = 5
relaxation_function = simple_power_law
tmax = time_fit[ind_fit.argmax()]
v0 = (app_height[-1] - app_height[0]) / (app_time[-1] - app_time[0])
#v0r = -1 * (ret_height[-1] - ret_height[0]) / (ret_time[-1] - ret_time[0])
slope = 0
f0 = 0
d0 = 0

In [ ]:
p0_ting_num = [E0, fluidity_exponent, f0, slope, d0]

resultTingNum = TingNumericalFit(
      ind_fit, force_fit, time_fit, indenter_shape, tip_parameter, p0_ting_num, dT, dTp,
      smoothing_window, relaxation_function, poisson_ratio
)

print(resultTingNum.fit_report())

In [ ]:
p0_ting_anal = [E0, fluidity_exponent, f0, slope, tmax]

resultTingAnal = TingAnaliticalFit(
     force_fit, time_fit, indenter_shape, tip_parameter, p0_ting_anal, dTp, v0, poisson_ratio
)

print(resultTingAnal.fit_report())

In [ ]:
plt.plot(ind_fit, force_fit)
plt.plot(ind_fit, resultTingNum.best_fit, '--', label='Ting Numercial best fit')
plt.plot(ind_fit, resultTingAnal.best_fit, '--', label='Ting Analytical best fit')
plt.xlabel("Indentation [m]")
plt.ylabel("Force[N]")
plt.legend()
plt.grid()
plt.show()

In [ ]:
np.abs(resultTingAnal.best_values['E0'] - resultTingNum.best_values['E0'])

In [ ]:
np.abs(resultTingAnal.best_values['beta'] - resultTingNum.best_values['alpha'])